In [44]:
!pip install keplergl==0.3.0
# !pip install nodejs
# !jupyter nbextension enable --py widgetsnbextension


# jupyter nbextension install --py --sys-prefix keplergl # can be skipped for notebook 5.3 and above

# jupyter nbextension enable --py --sys-prefix keplergl # can be skipped for notebook 5.3 and above

In [32]:
# !jupyter labextension install @jupyter-widgets/jupyterlab-manager keplergl-jupyter
!jupyter labextension install @jupyter-widgets/jupyterlab-manager@1.1


(Deprecated) Installing extensions with the jupyter labextension install command is now deprecated and will be removed in a future major version of JupyterLab.

Users should manage prebuilt extensions with package managers like pip and conda, and extension authors are encouraged to distribute their extensions as prebuilt packages 
An error occurred.
ValueError: 
"@jupyter-widgets/jupyterlab-manager@1.1.0" is not compatible with the current JupyterLab
Conflicting Dependencies:
JupyterLab                        Extension      Package
>=3.6.3 <3.7.0                    >=1.2.0 <2.0.0 @jupyterlab/application
>=5.6.3 <5.7.0                    >=3.2.0 <4.0.0 @jupyterlab/coreutils
>=3.6.3 <3.7.0                    >=1.0.0 <2.0.0 @jupyterlab/logconsole
>=3.6.3 <3.7.0                    >=1.2.0 <2.0.0 @jupyterlab/mainmenu
>=3.6.3 <3.7.0                    >=1.2.0 <2.0.0 @jupyterlab/notebook
>=3.6.3 <3.7.0                    >=1.2.0 <2.0.0 @jupyterlab/rendermime
>=3.6.3 <3.7.0                    

In [33]:
!jupyter labextension install keplergl-jupyter

(Deprecated) Installing extensions with the jupyter labextension install command is now deprecated and will be removed in a future major version of JupyterLab.

Users should manage prebuilt extensions with package managers like pip and conda, and extension authors are encouraged to distribute their extensions as prebuilt packages 
Building jupyterlab assets (production, minimized)


In [21]:
!jupyter nbextension install --py --sys-prefix keplergl # can be skipped for notebook 5.3 and above
!jupyter nbextension enable --py --sys-prefix keplergl # can be skipped for notebook 5.3 and above

Installing /Users/devlee/opt/anaconda3/lib/python3.9/site-packages/keplergl/static -> keplergl-jupyter
Up to date: /Users/devlee/opt/anaconda3/share/jupyter/nbextensions/keplergl-jupyter/keplergl.html
Up to date: /Users/devlee/opt/anaconda3/share/jupyter/nbextensions/keplergl-jupyter/index.js
Up to date: /Users/devlee/opt/anaconda3/share/jupyter/nbextensions/keplergl-jupyter/extension.js.map
Up to date: /Users/devlee/opt/anaconda3/share/jupyter/nbextensions/keplergl-jupyter/main.js
Up to date: /Users/devlee/opt/anaconda3/share/jupyter/nbextensions/keplergl-jupyter/index.js.map
Up to date: /Users/devlee/opt/anaconda3/share/jupyter/nbextensions/keplergl-jupyter/extension.js
- Validating: OK

    To initialize this nbextension in the browser every time the notebook (or other app) loads:
    
          jupyter nbextension enable keplergl --py --sys-prefix
    
Enabling notebook extension keplergl-jupyter/extension...
      - Validating: OK


In [34]:
!jupyter labextension install keplergl-jupyter

(Deprecated) Installing extensions with the jupyter labextension install command is now deprecated and will be removed in a future major version of JupyterLab.

Users should manage prebuilt extensions with package managers like pip and conda, and extension authors are encouraged to distribute their extensions as prebuilt packages 
Building jupyterlab assets (production, minimized)


In [30]:
!jupyter kernelspec list

Available kernels:
  python3    /Users/devlee/opt/anaconda3/share/jupyter/kernels/python3


In [54]:
import pandas as pd
import numpy as np
from keplergl import KeplerGl 
import json

In [4]:
call_taxi_df = pd.read_csv('./data/call_taxi.csv')

In [6]:
def convert_korean_datetime(custom_time_str: str):
    try:
        if '오전' in custom_time_str:
            custom_time_str = custom_time_str.replace('오전', 'AM')
        elif '오후' in custom_time_str:
            custom_time_str = custom_time_str.replace('오후', 'PM')
        return pd.to_datetime(custom_time_str, format='%Y-%m-%d %p %I:%M:%S')
    except Exception as e:
        return pd.NaT
call_taxi_df['settime_kor'] = call_taxi_df['settime'].apply(convert_korean_datetime)
call_taxi_df['ridetime_kor'] = call_taxi_df['ridetime'].apply(convert_korean_datetime)

call_taxi_df['대기시간'] = call_taxi_df['ridetime_kor'] - call_taxi_df['settime_kor']

def format_time_difference(td):
    if pd.isna(td):
        return 'N/A'
    minutes, seconds = divmod(td.total_seconds(), 60)
    return f"{int(minutes)}분 {int(seconds)}초"

call_taxi_df['대기시간'] = call_taxi_df['대기시간'].apply(format_time_difference)

def convert_time_to_seconds(time_str):
    parts = time_str.split(' ')
    if 'N/A' in parts: return 0
    minutes = int(parts[0][:-1]) if '분' in parts[0] else 0
    seconds = int(parts[1][:-1]) if '초' in parts[1] else 0
    return minutes * 60 + seconds

call_taxi_df['대기시간(초단위)'] = call_taxi_df['대기시간'].apply(convert_time_to_seconds)

In [7]:
def clean_startpos2(value):
    import re
    # Regular expression to match '제' followed by digits
    value = re.sub(r'제(\d+)', r'\1', value)
    return value



# 평균 대기시간 구하기
call_taxi_df['지역'] = call_taxi_df["startpos1"] + " " + call_taxi_df["startpos2"].apply(clean_startpos2)

average_time_per_startpoint = call_taxi_df.groupby('지역')['대기시간(초단위)'].mean().reset_index()

df_avg_wait_time = average_time_per_startpoint.rename(columns={'startpos2': 'startpos2', '대기시간(초단위)': '대기시간(초단위)'})


df_avg_wait_time['name'] = df_avg_wait_time['지역']
df_avg_wait_time['value'] = df_avg_wait_time['대기시간(초단위)']

In [52]:
geo_path = './data/seoul.geojson'
geo_json_data = json.load(open(geo_path, encoding='utf-8'))

temp_to_coords = {}
for feature in geo_json_data['features']:
    temp = feature['properties']['temp']
    coords = feature['geometry']['coordinates'][0][0][0]  # Use the first coordinate
    temp_to_coords[temp] = coords
call_taxi_df['longitude'] = call_taxi_df['지역'].map(lambda x: temp_to_coords.get(x, [None, None])[0])
call_taxi_df['latitude'] = call_taxi_df['지역'].map(lambda x: temp_to_coords.get(x, [None, None])[1])

call_taxi_df.to_csv('../../taxi.csv')

In [56]:
# NaN 값을 0으로 대체하거나 필요한 경우 다른 값으로 대체할 수 있습니다.
call_taxi_df.fillna(0, inplace=True)

# Infinity 값 검사 및 제거
call_taxi_df.replace([np.inf, -np.inf], np.nan, inplace=True)
call_taxi_df.dropna(inplace=True)

map_1 = KeplerGl(height=500)
map_1.add_data(call_taxi_df)
map_1

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter


KeplerGl(data={'unnamed': {'index': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20,…

In [41]:
pip install ipywidgets


Note: you may need to restart the kernel to use updated packages.


In [43]:
!jupyter nbextension list


Known nbextensions:
  config dir: /Users/devlee/.jupyter/nbconfig
    notebook section
      jupyter-js-widgets/extension  enabled 
      - Validating: OK
  config dir: /Users/devlee/opt/anaconda3/etc/jupyter/nbconfig
    notebook section
      jupyterlab-plotly/extension  enabled 
      - Validating: OK
      keplergl-jupyter/extension  enabled 
      - Validating: OK
      pydeck/extension  enabled 
      - Validating: problems found:
        - require?  X pydeck/extension
      jupyter-js-widgets/extension  enabled 
      - Validating: OK
